# BioGrid

Author: Moshe C. Silverstein  
Date: 07-2017

In [1]:
import sys
import numpy as np
import pandas as pd
import Utilities
from Utilities import *
from importlib import reload

# Load Data

Data Retrieved 1/25/2017  

https://thebiogrid.org/downloads/archives/Release%20Archive/BIOGRID-3.4.144/BIOGRID-ALL-3.4.144.mitab.zip

In [2]:
biogrid_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Input/BIOGRID-ALL-3.4.144.mitab.txt', sep='\t', index_col=False)

# Load Gene Mapping File

In [3]:
mappingFile = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Doc and Mapping/mappingFile_2017.txt', sep='\t', header=None, index_col=0)

# Select Relevent Data

In [4]:
# get only human and mouse data
n = biogrid_df['Taxid Interactor A'].values
b = [i for i,item in enumerate(n) if "taxid:9606" in item or "taxid:10090" in item] #taxid:9606 for human, taxid:10090 for mouse
biogrid_df = biogrid_df.ix[b]

biogrid_df =  biogrid_df.reset_index().drop('index', axis=1)

n = biogrid_df['Taxid Interactor B'].values
b = [i for i,item in enumerate(n) if "taxid:9606" in item or "taxid:10090" in item] #taxid:9606 for human, taxid:10090 for mouse
biogrid_df = biogrid_df.ix[b]

In [5]:
# select only relevent data
biogrid_df = biogrid_df[['Alt IDs Interactor A', 'Alt IDs Interactor B', 'Publication Identifiers', 'Source Database']]

In [6]:
# rename columns for consistency
biogrid_df.rename(columns={'Alt IDs Interactor A': 'Protein A (gene name)', 'Alt IDs Interactor B':'Protein B (gene name)', 'Publication Identifiers':'PubMed ID', 'Source Database':'Source databases'}, inplace=True)

In [7]:
# change columns to just show gene name

lstA = []
lstB = []

for i,index in enumerate(biogrid_df.index):
    
    progressPercent = ((i+1)/len(biogrid_df['Protein A (gene name)']))*100

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(biogrid_df['Protein A (gene name)'])))
    sys.stdout.flush()

    
    nameA = biogrid_df.ix[index, 'Protein A (gene name)']
    nameA = str(nameA) 
    if nameA != 'nan':
        lstA.append(nameA.split('|')[1].split(':')[1])
    else:
        lstA.append(np.nan)
        
    nameB = biogrid_df.ix[index, 'Protein B (gene name)']
    nameB = str(nameB) 
    if nameB != 'nan':
        lstB.append(nameB.split('|')[1].split(':')[1])
    else:
        lstB.append(np.nan)

biogrid_df['Protein A (gene name)'] = lstA
biogrid_df['Protein B (gene name)'] = lstB

In [8]:
biogrid_df.head()

# Map Gene to human and updated approved symbols

In [9]:
mapgenesymbols(biogrid_df, mappingFile, 'Protein A (gene name)', 'Protein B (gene name)')

In [10]:
biogrid_df.shape

(339802, 4)

# Get PubMed Ids

In [11]:
# Get Only PubMed ID for publication identifier
getPubMedIds(biogrid_df, 'PubMed ID')

# Unfiltered

In [12]:
# uf = unfiltered
biogrid_uf_df = biogrid_df.copy()

### Combine duplicate ppis while concatanatig referances

In [13]:
combineDupPPIs(biogrid_uf_df)

### Create .sig File

In [14]:
pathU = '~/./Documents/Harmonizome/PPI Library/Output/IndividualResources/Unfiltered/'

createSigFile(biogrid_uf_df, pathU, 'biogrid', False)

# Filtered

In [15]:
biogrid_f_df = biogrid_df.copy()

In [16]:
# drop any data that was published with more then 10 PPI's per publication or doesnt have a ppi
filterPPIbyPubmed(biogrid_f_df, 10)

### Combine duplicate ppis while concatanatig referances

In [17]:
combineDupPPIs(biogrid_f_df)

## Create .sig File

In [18]:
pathF = '~/./Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/'

createSigFile(biogrid_f_df, pathF, 'biogrid', True)